In [1]:
import pandas as pd
import numpy as np
import math
import os
from matplotlib import pyplot as plt
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run UtilsLightGBMFingerPrinting.ipynb

In [2]:
FILES = [
    
    #k = 1 SVR
    './FinalResult/Resultados_Metodo_13.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_14.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_15.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 2 SVR
    './FinalResult/Resultados_Metodo_16.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_17.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_18.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 3 SVR
    './FinalResult/Resultados_Metodo_19.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_20.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_21.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 4 SVR
    './FinalResult/Resultados_Metodo_22.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_23.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_24.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 5 SVR
    './FinalResult/Resultados_Metodo_25.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_26.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_27.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    
]
METHODS = [x.split("_")[-1].split(".")[0] for x in FILES]     # lista contendo os metodos que serão usados
db = pd.read_csv('databases/meds_pandas.csv')
erbs = pd.read_csv('databases/Bts.csv')
#errorLocation = pd.read_csv('result/erros.csv')
ERROS_CSV_FILE = './result-k-fold/erros-SVR-k-fold.csv' 

In [3]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,49.79,74.60,63.25,52.18,75.12,63.75,52.61,89.78,69.76,47.12,71.44,62.35,48.39,75.05,63.03
std,41.51,37.61,42.90,45.87,37.96,44.03,44.27,24.31,41.47,43.21,36.64,40.34,41.75,38.29,40.80
min,0.90,17.44,1.82,2.55,17.44,1.87,1.02,32.13,0.44,0.86,17.44,0.74,0.65,17.44,0.94
25%,16.52,42.13,26.57,16.92,42.13,24.43,17.87,72.88,32.51,14.91,42.13,29.76,15.79,42.13,29.20
50%,37.07,87.04,58.30,37.83,87.04,57.23,39.11,87.04,72.88,35.62,68.72,62.68,32.90,87.04,59.64
75%,74.24,102.64,97.92,75.22,102.64,97.92,77.23,102.64,102.06,66.25,102.64,92.04,70.70,102.64,96.42
max,189.02,137.37,225.27,258.61,137.37,224.26,237.98,137.37,217.83,249.75,137.37,163.26,184.93,198.83,196.48


# SVR-Outdoor:

In [4]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df
mean_outdoor = (test_erros['13'] + test_erros['16'] + test_erros['19'] + test_erros['22'] + test_erros['25'])/5
mean_outdoor.head()

0    72.008
1    41.732
2    40.584
3    31.434
4    18.888
dtype: float64

In [5]:
min_outdoor = min(test_erro['13']['min'], test_erro['16']['min'], test_erro['19']['min'], test_erro['22']['min'], test_erro['25']['min'])
min_outdoor

0.65

In [6]:
max_outdoor = max(test_erro['13']['max'], test_erro['16']['max'], test_erro['19']['max'], test_erro['22']['max'], test_erro['25']['max'])
max_outdoor

258.61

# SVR-Indoor:

In [7]:
mean_indoor = (test_erros['14'] + test_erros['17'] + test_erros['20'] + test_erros['23'] + test_erros['26'])/5
mean_indoor.head()

0     87.040
1     42.130
2    112.330
3     28.528
4     72.414
dtype: float64

In [8]:
min_indoor = min(test_erro['14']['min'], test_erro['17']['min'], test_erro['20']['min'], test_erro['23']['min'], test_erro['26']['min'])
min_indoor

17.44

In [9]:
max_indoor = max(test_erro['14']['max'], test_erro['17']['max'], test_erro['20']['max'], test_erro['23']['max'], test_erro['26']['max'])
max_indoor

198.83

# SVR-Indoor-Outdoor:

In [10]:
mean_indoor_outdoor = (test_erros['15'] + test_erros['18'] + test_erros['21'] + test_erros['24'] + test_erros['27'])/5
mean_indoor_outdoor.head()

0    55.760
1    23.452
2    62.560
3    79.168
4    82.040
dtype: float64

In [11]:
min_indoor_outdoor = min(test_erro['15']['min'], test_erro['18']['min'], test_erro['21']['min'], test_erro['24']['min'], test_erro['27']['min'])
min_indoor_outdoor

0.44

In [12]:
max_indoor_outdoor = max(test_erro['15']['max'], test_erro['18']['max'], test_erro['21']['max'], test_erro['24']['max'], test_erro['27']['max'])
max_indoor_outdoor

225.27

In [13]:
#errors_df.to_csv(ERROS_CSV_FILE, index=False)

In [17]:
erros_SVR = [mean_outdoor,mean_indoor, mean_indoor_outdoor ]
erros_SVR = pd.DataFrame(erros_SVR)
erros_SVR = erros_SVR.T
erros_SVR = round(erros_SVR,2)
erros_SVR.to_csv(ERROS_CSV_FILE, index=False)
erros_SVR.head()


,0,1,2
0,72.01,87.04,55.76
1,41.73,42.13,23.45
2,40.58,112.33,62.56
3,31.43,28.53,79.17
4,18.89,72.41,82.04


In [15]:
results_SVR = round(erros_SVR.describe(), 2)
results_SVR
#results_SVR.to_csv('./result-k-fold/Result-SVR-k-fold.csv', index=False)

,0,1,2
count,400.00,400.00,400.00
mean,50.02,77.20,64.43
std,18.17,33.90,31.16
min,8.39,28.53,11.20
25%,37.14,42.13,42.04
50%,48.42,87.04,55.75
75%,61.26,102.64,87.04
max,106.63,137.37,137.37
